In [5]:
!pip install flask

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 2.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.3/242.3 kB 3.0 MB/s eta 0:00:00a 0:00:01


In [12]:
from flask import Flask, request, jsonify
from pyspark.sql import SparkSession
import os

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressionModel

# Create a new SparkSession
spark = SparkSession.builder.appName("FlaskApp").getOrCreate()
model = RandomForestRegressionModel.load("randomForestpredictionmodel")
print("Loaded Model!")

# Prediction Function
def predict_volume(model, input_data):
    # Assemble the features into a vector column
    assembler = VectorAssembler(inputCols=['vol_moving_avg', 'adj_close_rolling_med'], outputCol="features")
    input_data = assembler.transform(input_data)

    # Make predictions on the new data
    predictions = model.transform(input_data)

    # Select relevant columns and return the DataFrame
    output = predictions.select('prediction')
    return output

app = Flask(__name__)

@app.route('/')
@app.route('/health')
def health():
    return jsonify({"message":'App is healthy!'})

@app.route('/predict', methods=['GET'])
def predict():
    global model
    # Get the values of the 'vol_moving_avg' and 'adj_close_rolling_med' query parameters
    vol_moving_avg = float(request.args.get('vol_moving_avg'))
    adj_close_rolling_med = float(request.args.get('adj_close_rolling_med'))

    input_data = spark.createDataFrame([(vol_moving_avg, adj_close_rolling_med)], ['vol_moving_avg', 'adj_close_rolling_med'])

    predictions = predict_volume(model, input_data)
    # Convert the predictions to a JSON response
    output = {
        "input": {
            "vol_moving_avg": vol_moving_avg, 
            "adj_close_rolling_med": adj_close_rolling_med
        },
        'predictions': predictions.select('prediction').collect()
    }
    return jsonify(output)


Loaded Model!


In [ ]:
app.run(host="0.0.0.0",port=8000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.17.0.2:8000
Press CTRL+C to quit
